In [6]:
import os
import re

import emoji
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# tf and keras
import tensorflow as tf
from tensorflow.keras import Sequential, layers, losses
from tensorflow.keras.layers import (
    Dense,
    Embedding,
    GlobalAveragePooling1D,
    Dropout,
    TextVectorization,
    Input,
    Conv1D,
    LSTM,
    MaxPooling1D,
    Bidirectional,
)
from tensorflow.keras.models import Model
# import tensorflow_datasets as tfds

pd.set_option('display.max_colwidth', 100) 

# UTILS

In [59]:
X_train = pd.read_csv('./data/final/X_train.csv')
y_train = pd.read_csv('./data/final/y_train.csv')
X_val = pd.read_csv('./data/final/X_val.csv')
y_val = pd.read_csv('./data/final/y_val.csv')
X_test = pd.read_csv('./data/final/X_test.csv')
y_test = pd.read_csv('./data/final/y_test.csv')

In [61]:
def read_files():
    X_train = pd.read_csv('./data/final/X_train.csv')
    y_train = pd.read_csv('./data/final/y_train.csv')
    X_val = pd.read_csv('./data/final/X_val.csv')
    y_val = pd.read_csv('./data/final/y_val.csv')
    X_test = pd.read_csv('./data/final/X_test.csv')
    y_test = pd.read_csv('./data/final/y_test.csv')
    
    train_not_na_indices = (X_train['fulltext'].notna())
    val_not_na_indices = (X_val['fulltext'].notna())
    test_not_na_indices = (X_test['fulltext'].notna())
    
    X_train = X_train[train_not_na_indices]
    X_val = X_val[val_not_na_indices]
    X_test = X_test[test_not_na_indices]
    
    y_train = y_train[train_not_na_indices]
    y_val = y_val[val_not_na_indices]
    y_test = y_test[test_not_na_indices]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [62]:
def get_vectorization_layer(df, column, max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    vectorize_layer = layers.TextVectorization(
        max_tokens=max_tokens,
        output_mode='int',
        output_sequence_length=output_sequence_length)

    df[column] = df[column].astype(str)
    vectorize_layer.adapt(df[column].values)

    return vectorize_layer

In [73]:
X_train, y_train, X_val, y_val, X_test, y_test = read_files()

### Convert the fulltext into tensors

In [74]:
text_data = tf.constant(X_train['fulltext'].values)
text_data_val = tf.constant(X_val['fulltext'].values)
text_data_test = tf.constant(X_test['fulltext'].values)

### Get vectorization layer

In [75]:
vectorize_layer = get_vectorization_layer(X_train, 'fulltext')

### Vectorize Text

In [76]:
vectorized_text = vectorize_layer(text_data)
vectorized_text_val = vectorize_layer(text_data_val)
vectorized_text_test = vectorize_layer(text_data_test)

# Model 2: TextVectorization Layer, Basic Embedding Model, Two Hidden Dense Layers (64, 32)

In [83]:
def build_model_2(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)

    x = GlobalAveragePooling1D()(x)

    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [84]:
model_2 = build_model_2()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_2 = model_2.fit(vectorized_text, y_train, epochs=5, batch_size=2,verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
pd.DataFrame(history_2.history)

Epoch 1/5
14787/14787 [==============================] - 344s 23ms/step - loss: 3367776.7500 - mean_absolute_error: 207.9170 - accuracy: 0.0230 - val_loss: 2434559.0000 - val_mean_absolute_error: 214.4482 - val_accuracy: 0.0229
Epoch 2/5
14787/14787 [==============================] - 283s 19ms/step - loss: 3277985.2500 - mean_absolute_error: 199.6094 - accuracy: 0.0231 - val_loss: 2298261.5000 - val_mean_absolute_error: 218.9449 - val_accuracy: 0.0229
Epoch 3/5
14787/14787 [==============================] - 280s 19ms/step - loss: 3095563.0000 - mean_absolute_error: 191.3588 - accuracy: 0.0231 - val_loss: 2159502.2500 - val_mean_absolute_error: 173.8821 - val_accuracy: 0.0229
Epoch 4/5
14787/14787 [==============================] - 325s 22ms/step - loss: 2912115.0000 - mean_absolute_error: 179.4832 - accuracy: 0.0231 - val_loss: 2122973.2500 - val_mean_absolute_error: 196.6857 - val_accuracy: 0.0229
Epoch 5/5
14787/14787 [==============================] - 393s 27ms/step - loss: 2808589.

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,3367776.75,207.917023,0.022959,2434559.00,214.448196,0.022881
1,3277985.25,199.609390,0.023095,2298261.50,218.944931,0.022881
2,3095563.00,191.358780,0.023095,2159502.25,173.882080,0.022881
3,2912115.00,179.483231,0.023095,2122973.25,196.685654,0.022881
4,2808589.25,180.011963,0.022959,2139633.00,191.864243,0.022881


In [13]:
model_2.save("model2.h5")

# Model 3: TextVectorization Layer, Convolutional NN Embedding Model, Two Hidden Dense Layers (64, 32)

In [21]:
def build_model_3(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)

    x = layers.Conv1D(32, 4, activation='relu')(x)
    x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [22]:
model_3 = build_model_3()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_3 = model_3.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
pd.DataFrame(history_3.history)

Epoch 1/5
14787/14787 [==============================] - 349s 24ms/step - loss: 3368448.7500 - mean_absolute_error: 215.2397 - accuracy: 0.0230 - val_loss: 2432068.7500 - val_mean_absolute_error: 223.1521 - val_accuracy: 0.0229
Epoch 2/5
14787/14787 [==============================] - 388s 26ms/step - loss: 3295401.5000 - mean_absolute_error: 218.1068 - accuracy: 0.0231 - val_loss: 2371435.2500 - val_mean_absolute_error: 240.1999 - val_accuracy: 0.0229
Epoch 3/5
14787/14787 [==============================] - 364s 25ms/step - loss: 3261372.2500 - mean_absolute_error: 219.5644 - accuracy: 0.0231 - val_loss: 2365950.7500 - val_mean_absolute_error: 209.2107 - val_accuracy: 0.0229
Epoch 4/5
14787/14787 [==============================] - 375s 25ms/step - loss: 3252968.7500 - mean_absolute_error: 209.9308 - accuracy: 0.0231 - val_loss: 2366330.7500 - val_mean_absolute_error: 235.8703 - val_accuracy: 0.0229


,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,3368448.75,215.239716,0.022992,2432068.75,223.152115,0.022881
1,3295401.50,218.106750,0.023095,2371435.25,240.199860,0.022881
2,3261372.25,219.564392,0.023095,2365950.75,209.210663,0.022881
3,3252968.75,209.930786,0.023095,2366330.75,235.870300,0.022881


In [23]:
model_3.save("model3.h5")

In [91]:
def build_model_3_dropout(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)

    x = layers.Conv1D(32, 4, activation='relu')(x)
    x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [92]:
model_3_dp = build_model_3_dropout()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_3_dp = model_3_dp.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
pd.DataFrame(history_3_dp.history)

Epoch 1/5
14787/14787 [==============================] - 413s 28ms/step - loss: 3369879.2500 - mean_absolute_error: 208.5753 - accuracy: 0.0230 - val_loss: 2433884.0000 - val_mean_absolute_error: 212.0740 - val_accuracy: 0.0229
Epoch 2/5
14787/14787 [==============================] - 815s 55ms/step - loss: 3297194.5000 - mean_absolute_error: 211.0922 - accuracy: 0.0231 - val_loss: 2369031.2500 - val_mean_absolute_error: 230.5923 - val_accuracy: 0.0229
Epoch 3/5
14787/14787 [==============================] - 444s 30ms/step - loss: 3269741.0000 - mean_absolute_error: 209.4527 - accuracy: 0.0231 - val_loss: 2365194.0000 - val_mean_absolute_error: 204.3584 - val_accuracy: 0.0229
Epoch 4/5
14787/14787 [==============================] - 478s 32ms/step - loss: 3257827.7500 - mean_absolute_error: 204.6521 - accuracy: 0.0231 - val_loss: 2361781.5000 - val_mean_absolute_error: 227.6711 - val_accuracy: 0.0229
Epoch 5/5
14787/14787 [==============================] - 409s 28ms/step - loss: 3244544.

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,3369879.25,208.575317,0.022962,2433884.00,212.073975,0.022881
1,3297194.50,211.092178,0.023094,2369031.25,230.592270,0.022881
2,3269741.00,209.452713,0.023095,2365194.00,204.358353,0.022881
3,3257827.75,204.652115,0.023095,2361781.50,227.671066,0.022881
4,3244544.50,213.821579,0.023095,2377978.25,223.073090,0.022881


In [93]:
model_3_dp.save("model3_dp.h5")

# Model 4: TextVectorization Layer, LSTM RNN Embedding Model, Two Hidden Dense Layers (64, 32)

In [24]:
X_train, y_train, X_val, y_val = read_files()

In [25]:
def build_model_4(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)
    
    x = layers.LSTM(32, activation='relu')(x)
    # x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [26]:
model_4 = build_model_4()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_4 = model_4.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
pd.DataFrame(history_4.history)

Epoch 1/5
14787/14787 [==============================] - 3693s 250ms/step - loss: nan - mean_absolute_error: nan - accuracy: 0.0087 - val_loss: nan - val_mean_absolute_error: nan - val_accuracy: 0.0098
Epoch 2/5
14787/14787 [==============================] - 3723s 252ms/step - loss: nan - mean_absolute_error: nan - accuracy: 0.0088 - val_loss: nan - val_mean_absolute_error: nan - val_accuracy: 0.0098


,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,NaN,NaN,0.008724,NaN,NaN,0.009784
1,NaN,NaN,0.008758,NaN,NaN,0.009784


In [27]:
model_4.save("model4.h5")

In [38]:
def build_model_4_tanh(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)
    
    x = layers.LSTM(32, activation='tanh')(x)
    # x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='tanh')(x)
    x = Dense(32, activation='tanh')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])

    return model

In [39]:
model_4_tanh = build_model_4_tanh()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_4_tanh = model_4_tanh.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
pd.DataFrame(history_4_tanh.history)

Epoch 1/5


2024-07-29 17:23:55.153418: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-29 17:23:55.155824: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-29 17:23:55.157104: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14787/14787 [==============================] - ETA: 0s - loss: 3386512.0000 - mean_absolute_error: 151.2841 - accuracy: 0.0231

2024-07-29 18:22:35.366543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-29 18:22:35.369524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-29 18:22:35.371646: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14787/14787 [==============================] - 3722s 252ms/step - loss: 3386512.0000 - mean_absolute_error: 151.2841 - accuracy: 0.0231 - val_loss: 2474658.0000 - val_mean_absolute_error: 176.0317 - val_accuracy: 0.0229
Epoch 2/5
14787/14787 [==============================] - 3681s 249ms/step - loss: 3378709.0000 - mean_absolute_error: 170.9912 - accuracy: 0.0231 - val_loss: 2468527.5000 - val_mean_absolute_error: 195.7875 - val_accuracy: 0.0229
Epoch 3/5
14787/14787 [==============================] - 3692s 250ms/step - loss: 3375091.2500 - mean_absolute_error: 188.8635 - accuracy: 0.0231 - val_loss: 2465765.2500 - val_mean_absolute_error: 209.0116 - val_accuracy: 0.0229
Epoch 4/5
14787/14787 [==============================] - 3727s 252ms/step - loss: 3373447.2500 - mean_absolute_error: 199.2385 - accuracy: 0.0231 - val_loss: 2464347.0000 - val_mean_absolute_error: 217.9396 - val_accuracy: 0.0229
Epoch 5/5
14787/14787 [==============================] - 3729s 252ms/step - loss: 3372651.

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,3386512.00,151.284119,0.023061,2474658.00,176.031723,0.022881
1,3378709.00,170.991241,0.023095,2468527.50,195.787537,0.022881
2,3375091.25,188.863541,0.023095,2465765.25,209.011566,0.022881
3,3373447.25,199.238480,0.023095,2464347.00,217.939560,0.022881
4,3372651.50,208.318726,0.023095,2463587.00,223.824326,0.022881


In [40]:
model_4_tanh.save("model4_tanh.h5")

# Model 5: TextVectorization Layer, Bi-Directional LSTM RNN Embedding Model, Two Hidden Dense Layers (64, 32)

In [28]:
X_train, y_train, X_val, y_val = read_files()

In [29]:
def build_model_5(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)
    
    x = Bidirectional(LSTM(32, activation='relu'))(x)
    # x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [30]:
model_5 = build_model_5()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_5 = model_5.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
pd.DataFrame(history_5.history)

Epoch 1/5
14787/14787 [==============================] - 5422s 367ms/step - loss: nan - mean_absolute_error: nan - accuracy: 0.0088 - val_loss: nan - val_mean_absolute_error: nan - val_accuracy: 0.0098
Epoch 2/5
14787/14787 [==============================] - 8753s 592ms/step - loss: nan - mean_absolute_error: nan - accuracy: 0.0088 - val_loss: nan - val_mean_absolute_error: nan - val_accuracy: 0.0098


,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,NaN,NaN,0.008792,NaN,NaN,0.009784
1,NaN,NaN,0.008758,NaN,NaN,0.009784


In [31]:
model_5.save("model5.h5")

In [45]:
def build_model_5_tanh(max_tokens=10000, output_sequence_length=250, embedding_dim=16):
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    inputs = Input(shape=(output_sequence_length,))
    x = Embedding(input_dim=max_tokens, output_dim=embedding_dim, input_length=output_sequence_length)(inputs)
    
    x = Bidirectional(LSTM(32, activation='tanh'))(x)
    # x = layers.MaxPooling1D()(x)
    
    x = Dense(64, activation='tanh')(x)
    x = Dense(32, activation='tanh')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error','accuracy'])
    
    return model

In [46]:
model_5_tanh = build_model_5_tanh()
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
history_5_tanh = model_5_tanh.fit(vectorized_text, y_train, epochs=5, batch_size=2, verbose=1, callbacks=[early_stopping], validation_data=(vectorized_text_val, y_val))
pd.DataFrame(history_5_tanh.history)

2024-07-30 07:27:20.485222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-30 07:27:20.489091: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-30 07:27:20.492208: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5


2024-07-30 07:27:21.613308: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-30 07:27:21.617623: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-30 07:27:21.620539: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14787/14787 [==============================] - ETA: 0s - loss: 3387469.2500 - mean_absolute_error: 149.3703 - accuracy: 0.0231

2024-07-30 09:00:28.890732: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-30 09:00:28.894690: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-30 09:00:28.898878: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14787/14787 [==============================] - 6031s 407ms/step - loss: 3387469.2500 - mean_absolute_error: 149.3703 - accuracy: 0.0231 - val_loss: 2477051.7500 - val_mean_absolute_error: 170.6082 - val_accuracy: 0.0229
Epoch 2/5
14787/14787 [==============================] - 5521s 373ms/step - loss: 3381214.2500 - mean_absolute_error: 163.5030 - accuracy: 0.0231 - val_loss: 2471167.7500 - val_mean_absolute_error: 186.0786 - val_accuracy: 0.0229
Epoch 3/5
14787/14787 [==============================] - 5534s 374ms/step - loss: 3377568.0000 - mean_absolute_error: 178.1383 - accuracy: 0.0231 - val_loss: 2468094.0000 - val_mean_absolute_error: 197.6012 - val_accuracy: 0.0229
Epoch 4/5
14787/14787 [==============================] - 5550s 375ms/step - loss: 3375999.2500 - mean_absolute_error: 187.7027 - accuracy: 0.0231 - val_loss: 2466325.2500 - val_mean_absolute_error: 205.9690 - val_accuracy: 0.0229
Epoch 5/5
14787/14787 [==============================] - 5468s 370ms/step - loss: 3374496.

,loss,mean_absolute_error,accuracy,val_loss,val_mean_absolute_error,val_accuracy
0,3387469.25,149.370346,0.023061,2477051.75,170.608170,0.022881
1,3381214.25,163.502960,0.023095,2471167.75,186.078568,0.022881
2,3377568.00,178.138260,0.023095,2468094.00,197.601196,0.022881
3,3375999.25,187.702682,0.023095,2466325.25,205.969009,0.022881
4,3374496.75,195.912979,0.023095,2465304.75,211.702026,0.022881


In [47]:
model_5_tanh.save("model5_tanh.h5")

# Model Evaluations

In [113]:
def validate_model(model, x_train, y_train, x_val, y_val, x_test, y_test):
    train_loss_mse, train_mae, train_acc = model.evaluate(x_train, y_train, verbose=0)
    val_loss_mse, val_mae, val_acc = model.evaluate(x_val, y_val, verbose=0)
    test_loss_mse, test_mae, test_acc = model.evaluate(x_test, y_test, verbose=0)
    
    return {
        "train": {"mse":train_loss_mse,"mae":train_mae,"acc":train_acc},
        "val": {"mse":val_loss_mse,"mae":val_mae,"acc":val_acc},
        "test": {"mse":test_loss_mse,"mae":test_mae,"acc":test_acc},
    }

In [114]:
model_2_saved = tf.keras.models.load_model('model2.h5')
model_3_saved = tf.keras.models.load_model('model3.h5')
model_3_dropout_saved = tf.keras.models.load_model('model3_dp.h5')
model_4_tanh_saved = tf.keras.models.load_model('model4_tanh.h5')
model_5_tanh_saved = tf.keras.models.load_model('model5_tanh.h5')

2024-07-30 19:16:14.067278: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-30 19:16:14.070030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-30 19:16:14.071705: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [115]:
model_2_eval = validate_model(model_2_saved ,vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_3_eval = validate_model(model_3_saved, vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_3_dropout_eval = validate_model(model_3_dropout_saved,vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_4_tanh_eval = validate_model(model_4_tanh_saved, vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)
model_5_tanh_eval = validate_model(model_5_tanh_saved, vectorized_text, y_train, vectorized_text_val, y_val, vectorized_text_test, y_test)

2024-07-30 19:17:25.999311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-30 19:17:26.003657: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-30 19:17:26.005964: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [123]:
pd.DataFrame(model_2_eval).T

,mse,mae,acc
train,2778908.250,170.873810,0.021844
val,2129838.000,198.186859,0.020357
test,974498.625,158.912216,0.022405


In [124]:
pd.DataFrame(model_3_eval).T

,mse,mae,acc
train,3244034.500,196.161621,0.023095
val,2365950.250,209.210739,0.022881
test,1215377.875,162.270172,0.024771


In [125]:
pd.DataFrame(model_3_dropout_eval).T

,mse,mae,acc
train,3218350.5,214.189423,0.023095
val,2361781.5,227.670670,0.022881
test,1201115.0,180.888901,0.024771


In [126]:
pd.DataFrame(model_4_tanh_eval).T

,mse,mae,acc
train,3372429.500,210.041733,0.023095
val,2463589.000,223.823853,0.022881
test,1319618.875,174.204758,0.024771


In [127]:
pd.DataFrame(model_5_tanh_eval).T

,mse,mae,acc
train,3373637.750,197.654053,0.023095
val,2465303.250,211.701187,0.022881
test,1319630.375,161.669479,0.024771


Model 2 performed the best in terms of validation dataset loss, which calculated as the mean squared error.

Model 2 - Validation Loss after 5 Epochs - 2145500.25
Model 3 - Validation Loss after 3 Epochs - 2365950.75
Model 4 - Validation Loss after 4 Epochs - 2463587.00
Model 5 - Validation Loss after 5 Epochs - 2465304.75

In [52]:
display(test.layers)
display(test.summary())

# Retrieve the embeddings layer, which itself is wrapped in a list.
embeddings = test.layers[1].get_weights()[0]
print('-'*100)
display("Embeddings layer - shape: ", embeddings.shape)
print('-'*100)
display("Embeddings layer - parameter matrix (before training): ", embeddings)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 250)]             0         
                                                                 
 embedding (Embedding)       (None, 250, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                             

None

----------------------------------------------------------------------------------------------------


'Embeddings layer - shape: '

(10000, 16)

----------------------------------------------------------------------------------------------------


'Embeddings layer - parameter matrix (before training): '

array([[ 0.07392167, -0.01201833,  0.07342985, ...,  0.05041577,
         0.08608239,  0.05633584],
       [ 0.01863779, -0.01785451, -0.0067727 , ..., -0.02176556,
        -0.01128479,  0.01053058],
       [ 0.01540924, -0.04836084, -0.0321469 , ...,  0.01791335,
        -0.00249856, -0.00868337],
       ...,
       [ 0.0019884 ,  0.01765646, -0.01861403, ...,  0.01839873,
        -0.00617508, -0.02580006],
       [-0.01264708, -0.01301428, -0.02354966, ...,  0.02202214,
         0.01753282,  0.00889954],
       [ 0.01021462,  0.01071981,  0.04415278, ...,  0.00548404,
         0.01263346, -0.02413355]], dtype=float32)

# End of file

Sources:
* https://stackoverflow.com/questions/73878049/how-do-you-convert-the-pandas-dataframe-to-tensorflow-python-data-ops-dataset-op